In [5]:
from __future__ import print_function
import os
import sys
import re
import operator
import pandas as pd
import numpy as np

# Edit this to be the full path to the extracted Spark directory on your computer
spark_home = 'C:\Users\Vikaasa\IdeaProjects\Spark-Workshop\spark-1.6.1-bin-hadoop2.6'

os.environ['SPARK_HOME'] = spark_home
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python\lib\py4j-0.9-src.zip'))

In [6]:
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
users = sqlContext.read.json("yelp_academic_dataset_user.json")
users.count()

552339

In [4]:
users.printSchema()


root
 |-- average_stars: double (nullable = true)
 |-- compliments: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- cute: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- hot: long (nullable = true)
 |    |-- list: long (nullable = true)
 |    |-- more: long (nullable = true)
 |    |-- note: long (nullable = true)
 |    |-- photos: long (nullable = true)
 |    |-- plain: long (nullable = true)
 |    |-- profile: long (nullable = true)
 |    |-- writer: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- fans: long (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)

In [411]:
comments = sqlContext.read.json("yelp_academic_dataset_business.json")
comments.count()

77445

In [412]:
comments.printSchema()

root
 |-- attributes: struct (nullable = true)
 |    |-- Accepts Credit Cards: boolean (nullable = true)
 |    |-- Accepts Insurance: boolean (nullable = true)
 |    |-- Ages Allowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: struct (nullable = true)
 |    |    |-- casual: boolean (nullable = true)
 |    |    |-- classy: boolean (nullable = true)
 |    |    |-- divey: boolean (nullable = true)
 |    |    |-- hipster: boolean (nullable = true)
 |    |    |-- intimate: boolean (nullable = true)
 |    |    |-- romantic: boolean (nullable = true)
 |    |    |-- touristy: boolean (nullable = true)
 |    |    |-- trendy: boolean (nullable = true)
 |    |    |-- upscale: boolean (nullable = true)
 |    |-- Attire: string (nullable = true)
 |    |-- BYOB: boolean (nullable = true)
 |    |-- BYOB/Corkage: string (nullable = true)
 |    |-- By Appointment Only: boolean (nullable = true)
 |    |-- Caters: boolean (nullable = true)
 |    |-- Coat Chec

In [413]:
comments.show(3)

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----------+-----------+----------------+-------------+----+------------+-----+-----+--------+
|          attributes|         business_id|          categories|      city|        full_address|               hours|  latitude|  longitude|            name|neighborhoods|open|review_count|stars|state|    type|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----------+-----------+----------------+-------------+----+------------+-----+-----+--------+
|[true,null,null,n...|5UmKMjUEUNdYWqANh...|[Fast Food, Resta...|Dravosburg|4734 Lebanon Chur...|[[21:00,11:00],[2...|40.3543266|-79.9007057|       Mr Hoagie|           []|true|           4|  4.5|   PA|business|
|[true,null,null,n...|UsFtqoBl7naz8AVUB...|         [Nightlife]|Dravosburg|202 McClure St
Dr...|[null,null,null,n...|40.3505527|-79.8868138|    Clancy's Pub

In [414]:
comments.where(comments["state"]=="PA").count()


3754

In [556]:
business_PA=comments.where(comments["state"]=="PA")

In [557]:
business_PA.head()

Row(attributes=Row(Accepts Credit Cards=True, Accepts Insurance=None, Ages Allowed=None, Alcohol=u'none', Ambience=Row(casual=False, classy=False, divey=False, hipster=False, intimate=False, romantic=False, touristy=False, trendy=False, upscale=False), Attire=u'casual', BYOB=None, BYOB/Corkage=None, By Appointment Only=None, Caters=False, Coat Check=None, Corkage=None, Delivery=False, Dietary Restrictions=None, Dogs Allowed=None, Drive-Thru=False, Good For=Row(breakfast=False, brunch=False, dessert=False, dinner=False, latenight=False, lunch=False), Good For Dancing=None, Good For Groups=True, Good for Kids=True, Hair Types Specialized In=None, Happy Hour=None, Has TV=False, Music=None, Noise Level=u'average', Open 24 Hours=None, Order at Counter=None, Outdoor Seating=False, Parking=Row(garage=False, lot=False, street=False, valet=False, validated=False), Price Range=1, Smoking=None, Take-out=True, Takes Reservations=False, Waiter Service=False, Wheelchair Accessible=None, Wi-Fi=None),

In [558]:
b_PA_select = business_PA.select(business_PA.business_id,\
                business_PA.categories, business_PA.city, business_PA.state, \
                business_PA.latitude, business_PA.longitude, business_PA.name, business_PA.review_count, business_PA.stars, \
                business_PA.attributes.Alcohol, business_PA.attributes.Ambience.casual, business_PA.attributes.Ambience.classy, business_PA.attributes.Ambience.divey, \
                business_PA.attributes.Ambience.hipster, business_PA.attributes.Ambience.intimate, business_PA.attributes.Ambience.romantic, \
                business_PA.attributes.Ambience.touristy, business_PA.attributes.Ambience.trendy, business_PA.attributes.Ambience.upscale, \
                business_PA.attributes.Delivery, business_PA.neighborhoods, business_PA.open)

In [449]:
b_PA_pandas=b_PA_select.orderBy(b_PA_select.business_id).toPandas()

In [450]:
b_PA_pandas['business_id']=b_PA_pandas['business_id'].map(lambda x: x.lstrip('=+-'))

In [451]:
b_PA_pandas.to_csv("b_PA.csv", encoding='utf-8')

In [559]:
b_PA_pandas.head()

,business_id,categories,city,state,latitude,longitude,name,review_count,stars,attributes[Alcohol],...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,UE_y6auTgq3FXlvUMkbw,"[American (Traditional), Diners, Restaurants]",Pittsburgh,PA,40.456390,-79.941311,Ritters Diner,133,3.5,none,...,False,False,False,False,False,False,False,False,[Bloomfield],True
1,1lvWTeV_RI1weq8FTnPRw,"[Dive Bars, Bars, Nightlife]",Pittsburgh,PA,40.456824,-79.966497,Gooski's,66,4.5,full_bar,...,True,False,False,False,False,False,False,None,[Polish Hill],True
2,23t2qOQWj8Kqp-Ijm1BQA,"[Chinese, Restaurants]",Pittsburgh,PA,40.439226,-80.001725,Mandarin Gourmet,36,3.0,none,...,False,False,False,False,False,False,False,False,[Downtown],True
3,2pv5J-_HN1Tu8adfRxFlg,"[Electronics, Shopping]",Pittsburgh,PA,40.429025,-79.979677,Unlocked Global Phones,3,2.5,None,...,None,None,None,None,None,None,None,None,[South Side],True
4,4bNlx03GsXzgkHZwTrMaw,"[Bars, American (Traditional), Pubs, Nightlife...",Pittsburgh,PA,40.442324,-80.001985,Redbeard's On Sixth,83,3.0,full_bar,...,False,False,False,False,False,False,False,False,[Downtown],True


In [429]:
reviews = sqlContext.read.json("yelp_academic_dataset_review.json")
reviews.count()

2225213

In [1706]:
reviews.select(reviews.business_id, 
reviews.user_id, reviews.review_id, reviews.text, reviews.stars, reviews.date).show(n=3)

+--------------------+--------------------+--------------------+--------------------+-----+----------+
|         business_id|             user_id|           review_id|                text|stars|      date|
+--------------------+--------------------+--------------------+--------------------+-----+----------+
|5UmKMjUEUNdYWqANh...|PUFPaY9KxDAcGqfso...|Ya85v4eqdd6k9Od8H...|Mr Hoagie is an i...|    4|2012-08-01|
|5UmKMjUEUNdYWqANh...|Iu6AxdBYGR4A0wspR...|KPvLNJ21_4wbYNctr...|Excellent food. S...|    5|2014-02-13|
|5UmKMjUEUNdYWqANh...|auESFwWvW42h6alXg...|fFSoGV46Yxuwbr3fH...|Yes this place is...|    5|2015-10-31|
+--------------------+--------------------+--------------------+--------------------+-----+----------+
only showing top 3 rows



In [1707]:
reviews_joined=reviews.join(business_PA, reviews.business_id == business_PA.business_id, 'inner').select(reviews.business_id,business_PA.state, 
business_PA.review_count, reviews.user_id, reviews.review_id, reviews.text, reviews.stars, reviews.date)

In [1711]:
reviews_joined_again=users.join(reviews_joined, users.user_id == reviews_joined.user_id, 'inner').select(reviews_joined.business_id,reviews_joined.state,
reviews_joined.review_count, reviews_joined.user_id, reviews_joined.review_id, reviews_joined.text, reviews_joined.stars, 
users.review_count, users.fans, users.votes.useful, users.yelping_since, reviews_joined.date)

In [1713]:
reviews_joined_again.orderBy(reviews_joined_again.business_id).toPandas().to_csv("testreviews.csv", encoding='utf-8')

In [1714]:
reviews_joined_again=reviews_joined_again.orderBy(reviews_joined_again.business_id)

In [1715]:
reviews_pandas=reviews_joined_again.orderBy(reviews_joined_again.business_id).toPandas()

In [1716]:
reviews_pandas['business_id']=reviews_pandas['business_id'].map(lambda x: x.lstrip('=+-'))

In [1717]:
reviews_pandas['user_id']=reviews_pandas['user_id'].map(lambda x: x.lstrip('=+-'))

In [1718]:
reviews_pandas['review_id']=reviews_pandas['review_id'].map(lambda x: x.lstrip('=+-'))

In [1719]:
reviews_pandas.to_csv("reviews.csv", encoding='utf-8')

In [1720]:
reviews_pandas.head()

,business_id,state,review_count,user_id,review_id,text,stars,review_count,fans,votes[useful],yelping_since,date
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,Ritters is a Pittsburgh treasure. Old school d...,3,29,0,13,2014-01,2014-01-21
1,UE_y6auTgq3FXlvUMkbw,PA,133,Jqk1eCOQw1A7sE6DD4PRdw,GE9olaftv8AUq477WgTdQg,"Great food, great service. Only thing I don't ...",4,11,0,0,2015-12,2015-12-15
2,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,The food at Ritter's tastes like I made it at ...,2,250,13,678,2012-06,2012-12-22
3,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,Sometimes at four in the morning you need a pl...,3,131,4,220,2010-12,2011-08-23
4,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,"Ritter's is amazing- for what it is: A big, Gr...",3,48,3,94,2010-08,2011-04-17


In [1721]:
reviews.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- useful: long (nullable = true)



In [607]:

b_PA_pandas = pd.read_csv('b_PA.csv')
b_PA_pandas = b_PA_pandas.drop('Unnamed: 0', 1)


In [1391]:
trainingdata = pd.read_csv('reviews-remove6m.csv')
trainingdata = trainingdata.drop('Unnamed: 0', 1)

In [1392]:
traindata=trainingdata[trainingdata.after_Jun2015 == 0]

In [1393]:
traindata.to_csv("traindata.csv", encoding='utf-8')

In [1400]:
traindata = pd.read_csv('traindata.csv')
traindata = traindata.drop('Unnamed: 0', 1)

In [1405]:
traindata

,business_id,state,business_review_count,user_id,review_id,text,stars_review,user_review_count,fans,votes[useful],...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,Ritters is a Pittsburgh treasure. Old school d...,3,29,0,13,...,0.905000,2.715000,131.901667,440.610000,3.340443,0,0,0,1,1
1,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,The food at Ritter's tastes like I made it at ...,2,250,13,678,...,1.986667,3.973333,131.901667,440.610000,3.340443,0,0,0,0,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,Sometimes at four in the morning you need a pl...,3,131,4,220,...,1.133333,3.400000,131.901667,440.610000,3.340443,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,"Ritter's is amazing- for what it is: A big, Gr...",3,48,3,94,...,0.680000,2.040000,131.901667,440.610000,3.340443,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,"Oh, Ritters. I'd actually give this place a 3....",3,26,1,73,...,0.745000,2.235000,131.901667,440.610000,3.340443,0,0,0,0,1
5,UE_y6auTgq3FXlvUMkbw,PA,133,Ynt9QATr3tur66_OVQNCng,K9C4D2QLLjDAm2HbAC1H7Q,This is an institution. I hit the counter wit...,4,56,1,71,...,0.680000,2.720000,131.901667,440.610000,3.340443,0,0,0,0,1
6,UE_y6auTgq3FXlvUMkbw,PA,133,D1G8W0SgCbsKTv7cAqKcNg,0htxV_8Ylm_9rhkzAgFhXw,Ordered the 2 eggs and potatoes which was only...,3,397,13,823,...,2.731667,8.195000,131.901667,440.610000,3.340443,0,0,0,0,1
7,UE_y6auTgq3FXlvUMkbw,PA,133,WzaaorVCmUTQvu4mScunNg,_-PlgL53lTdVUcxc8K52Pw,"On a postive note, the honey dipped chicken di...",3,432,11,744,...,3.620000,10.860000,131.901667,440.610000,3.340443,0,0,1,1,1
8,UE_y6auTgq3FXlvUMkbw,PA,133,T_wjLgPOPXry7Bea4MzoVQ,csngrK1aFzljgD6MJWGkVQ,Ritter's will never be a fine dining establish...,4,808,38,2147,...,5.960000,23.840000,131.901667,440.610000,3.340443,0,0,0,0,1
9,UE_y6auTgq3FXlvUMkbw,PA,133,4nlTiVllEXvcvZow11khpA,f-PrxoPb687RvMGsRpWF7g,"Don't get me wrong, the foods great, but when ...",1,1,1,0,...,0.905000,0.905000,131.901667,440.610000,3.340443,0,0,0,1,1


In [1397]:
PA_newfeatures = traindata

In [1468]:
len(b_PA_pandas['business_id'])

3754

In [1469]:
len(idclean['business_id'])

3624

In [1470]:
len(join['business_id'])

2350

In [1471]:
len(join_tmp['business_id'])

76423

In [1472]:
len(trainingdata['business_id'])

92640

In [1473]:
PA_newfeatures.head()

,business_id,state,business_review_count,user_id,review_id,text,stars_review,user_review_count,fans,votes[useful],...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,Ritters is a Pittsburgh treasure. Old school d...,3,29,0,13,...,0.905000,2.715000,160.901667,531.276667,3.301872,0,0,0,1,1
1,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,The food at Ritter's tastes like I made it at ...,2,250,13,678,...,1.986667,3.973333,160.901667,531.276667,3.301872,0,0,0,0,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,Sometimes at four in the morning you need a pl...,3,131,4,220,...,1.133333,3.400000,160.901667,531.276667,3.301872,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,"Ritter's is amazing- for what it is: A big, Gr...",3,48,3,94,...,0.680000,2.040000,160.901667,531.276667,3.301872,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,"Oh, Ritters. I'd actually give this place a 3....",3,26,1,73,...,0.745000,2.235000,160.901667,531.276667,3.301872,0,0,0,0,1


In [1215]:
idclean=dict()

for i in range(len(testdata['business_id'])):
    if testdata['business_id'][i] not in idclean:
        idclean[testdata['business_id'][i]]=1
        
idclean=pd.DataFrame(idclean.items(), columns=['business_id','present'])
right=b_PA_pandas
left=idclean
join_test=left.merge(right, left_on='business_id', right_on='business_id', how='inner')


In [1216]:
cols = join_test.columns.tolist()
cols

['business_id',
 'present',
 'categories',
 'city',
 'state',
 'latitude',
 'longitude',
 'name',
 'review_count',
 'stars',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open',
 'users_sum',
 'ratings_avg']

In [1217]:
join_test=join_test[['business_id',
 'categories',
 'city',
 'state',
 'name',
 'review_count',
 'stars',
 'latitude',
 'longitude',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open']]

In [1474]:
idclean=dict()

for i in range(len(traindata['business_id'])):
    if traindata['business_id'][i] not in idclean:
        idclean[traindata['business_id'][i]]=1
        


In [1475]:
idclean=pd.DataFrame(idclean.items(), columns=['business_id','present'])

In [1476]:
idclean.head()

,business_id,present
0,shrMOQmX_SX-KualwYoAzA,1
1,c3GRR8q1h4fdNV7APLzSeQ,1
2,KxidRGk6RpyvVXibiy_DIg,1
3,OT3yVpDrDVITDCIYox8d9A,1
4,Verns0y0NTYj96SyvKgnhA,1


In [1477]:
right=b_PA_pandas
left=idclean

In [1478]:
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [1479]:
cols = join.columns.tolist()
cols

['business_id',
 'present',
 'categories',
 'city',
 'state',
 'latitude',
 'longitude',
 'name',
 'review_count',
 'stars',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open',
 'users_sum',
 'ratings_avg']

In [1480]:
join=join[['business_id',
 'categories',
 'city',
 'state',
 'name',
 'review_count',
 'stars',
 'latitude',
 'longitude',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open']]

In [1481]:
join.head()

,business_id,categories,city,state,name,review_count,stars,latitude,longitude,attributes[Alcohol],...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,shrMOQmX_SX-KualwYoAzA,"[Bakeries, Food]",Pittsburgh,PA,Au Bon Pain Co,7,3.0,40.441352,-79.998017,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,[Downtown],True
1,c3GRR8q1h4fdNV7APLzSeQ,"[Auto Repair, Automotive, Tires, Oil Change St...",Pittsburgh,PA,Homer's Service Center,40,4.5,40.423409,-79.928138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Greenfield],True
2,KxidRGk6RpyvVXibiy_DIg,"[Shopping, Home Services, Building Supplies, H...",Carnegie,PA,Lowe's,7,3.0,40.387855,-80.094976,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True
3,OT3yVpDrDVITDCIYox8d9A,"[Restaurants, Italian]",Pittsburgh,PA,Buon Giorno Cafe,5,3.5,40.438660,-80.005135,NaN,...,False,False,False,False,False,False,False,True,[Downtown],True
4,Verns0y0NTYj96SyvKgnhA,"[Optometrists, Ophthalmologists, Health & Medi...",Pittsburgh,PA,UPMC Eye Center,6,1.5,40.440987,-79.957777,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True


In [1803]:
trainingdata

,business_id,state,business_review_count,user_id,review_id,text,stars_review,user_review_count,fans,votes[useful],...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,Ritters is a Pittsburgh treasure. Old school d...,3,29,0,13,...,0.905000,2.715000,160.901667,531.276667,3.301872,0,0,0,1,1
1,UE_y6auTgq3FXlvUMkbw,PA,133,Jqk1eCOQw1A7sE6DD4PRdw,GE9olaftv8AUq477WgTdQg,"Great food, great service. Only thing I don't ...",4,11,0,0,...,1.000000,4.000000,160.901667,531.276667,3.301872,1,1,1,1,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,The food at Ritter's tastes like I made it at ...,2,250,13,678,...,1.986667,3.973333,160.901667,531.276667,3.301872,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,Sometimes at four in the morning you need a pl...,3,131,4,220,...,1.133333,3.400000,160.901667,531.276667,3.301872,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,"Ritter's is amazing- for what it is: A big, Gr...",3,48,3,94,...,0.680000,2.040000,160.901667,531.276667,3.301872,0,0,0,0,1
5,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,"Oh, Ritters. I'd actually give this place a 3....",3,26,1,73,...,0.745000,2.235000,160.901667,531.276667,3.301872,0,0,0,0,1
6,UE_y6auTgq3FXlvUMkbw,PA,133,Ynt9QATr3tur66_OVQNCng,K9C4D2QLLjDAm2HbAC1H7Q,This is an institution. I hit the counter wit...,4,56,1,71,...,0.680000,2.720000,160.901667,531.276667,3.301872,0,0,0,0,1
7,UE_y6auTgq3FXlvUMkbw,PA,133,D1G8W0SgCbsKTv7cAqKcNg,0htxV_8Ylm_9rhkzAgFhXw,Ordered the 2 eggs and potatoes which was only...,3,397,13,823,...,2.731667,8.195000,160.901667,531.276667,3.301872,0,0,0,0,1
8,UE_y6auTgq3FXlvUMkbw,PA,133,WzaaorVCmUTQvu4mScunNg,_-PlgL53lTdVUcxc8K52Pw,"On a postive note, the honey dipped chicken di...",3,432,11,744,...,3.620000,10.860000,160.901667,531.276667,3.301872,0,0,1,1,1
9,UE_y6auTgq3FXlvUMkbw,PA,133,T_wjLgPOPXry7Bea4MzoVQ,csngrK1aFzljgD6MJWGkVQ,Ritter's will never be a fine dining establish...,4,808,38,2147,...,5.960000,23.840000,160.901667,531.276667,3.301872,0,0,0,0,1


In [1482]:
after_Jan2015_grouped=traindata.groupby(['business_id'])['after_Jan2015'].sum().reset_index(name='after_Jan2015_count')

In [1483]:
after_Jan2015_grouped.head()

,business_id,after_Jan2015_count
0,01a4YOjiBbPUBxvgX6F7VQ,0
1,04ogq23JnzauhezrWEXG_g,3
2,06SQoq5EV4XAY-yKrEtCQg,0
3,079g3KpS_7j_Zw53JmD9Vg,5
4,08Z2A6u4kmaAZYeecwA4qw,5


In [1484]:
right=traindata
left=after_Jan2015_grouped

In [1485]:
join_tmp=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

cols = join_tmp.columns.tolist()
cols

In [1486]:
join_tmp=join_tmp[['business_id',
 'state',
 'business_review_count',
 'user_id',
 'review_id',
 'text',
 'stars_review',
 'user_review_count',
 'fans',
 'votes[useful]',
 'yelping_since',
 'date',
 'year_weighted',
 'review_count_weighted',
 'fans_weighted',
 'votes_weighted',
 'weighted_users',
 'weighted_rating',
 'users_sum',
 'ratings_sum',
 'ratings_avg',
 'after_Jun2015',
 'after_Jan2015',
 'after_Jun2014',
 'after_Jan2014',
 'after_Jan2010',
 'after_Jan2015_count']]

In [1487]:
after_Jun2014_grouped=traindata.groupby(['business_id'])['after_Jun2014'].sum().reset_index(name='after_Jun2014_count')

In [1488]:
after_Jun2014_grouped.head()

,business_id,after_Jun2014_count
0,01a4YOjiBbPUBxvgX6F7VQ,2
1,04ogq23JnzauhezrWEXG_g,9
2,06SQoq5EV4XAY-yKrEtCQg,0
3,079g3KpS_7j_Zw53JmD9Vg,14
4,08Z2A6u4kmaAZYeecwA4qw,6


In [1489]:
right=join_tmp
left=after_Jun2014_grouped

In [1490]:
join_tmp=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [1491]:
join_tmp=join_tmp[['business_id',
 'state',
 'business_review_count',
 'user_id',
 'review_id',
 'text',
 'stars_review',
 'user_review_count',
 'fans',
 'votes[useful]',
 'yelping_since',
 'date',
 'year_weighted',
 'review_count_weighted',
 'fans_weighted',
 'votes_weighted',
 'weighted_users',
 'weighted_rating',
 'users_sum',
 'ratings_sum',
 'ratings_avg',
 'after_Jun2015',
 'after_Jan2015',
 'after_Jun2014',
 'after_Jan2014',
 'after_Jan2010',
 'after_Jan2015_count',
 'after_Jun2014_count']]

In [1492]:
after_Jan2014_grouped=traindata.groupby(['business_id'])['after_Jan2014'].sum().reset_index(name='after_Jan2014_count')

In [1493]:
right=join_tmp
left=after_Jan2014_grouped

In [1494]:
join_tmp=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [1495]:
join_tmp=join_tmp[['business_id',
 'state',
 'business_review_count',
 'user_id',
 'review_id',
 'text',
 'stars_review',
 'user_review_count',
 'fans',
 'votes[useful]',
 'yelping_since',
 'date',
 'year_weighted',
 'review_count_weighted',
 'fans_weighted',
 'votes_weighted',
 'weighted_users',
 'weighted_rating',
 'users_sum',
 'ratings_sum',
 'ratings_avg',
 'after_Jun2015',
 'after_Jan2015',
 'after_Jun2014',
 'after_Jan2014',
 'after_Jan2010',
 'after_Jan2015_count',
 'after_Jun2014_count',
 'after_Jan2014_count']]

In [1496]:
after_Jan2010_grouped=trainingdata.groupby(['business_id'])['after_Jan2010'].sum().reset_index(name='after_Jan2010_count')
right=join_tmp
left=after_Jan2010_grouped
join_tmp=left.merge(right, left_on='business_id', right_on='business_id', how='outer')
join_tmp=join_tmp[['business_id',
 'state',
 'business_review_count',
 'user_id',
 'review_id',
 'text',
 'stars_review',
 'user_review_count',
 'fans',
 'votes[useful]',
 'yelping_since',
 'date',
 'year_weighted',
 'review_count_weighted',
 'fans_weighted',
 'votes_weighted',
 'weighted_users',
 'weighted_rating',
 'users_sum',
 'ratings_sum',
 'ratings_avg',
 'after_Jun2015',
 'after_Jan2015',
 'after_Jun2014',
 'after_Jan2014',
 'after_Jan2010',
 'after_Jan2015_count',
 'after_Jun2014_count',
 'after_Jan2014_count',
 'after_Jan2010_count']]

In [1166]:
join_tmp

,business_id,state,business_review_count,user_id,review_id,text,stars_review,user_review_count,fans,votes[useful],...,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010,after_Jan2015_count,after_Jun2014_count,after_Jan2014_count,after_Jan2010_count
0,01a4YOjiBbPUBxvgX6F7VQ,PA,3,RJChsLqZrY3DRnaFbTXVXw,BLpxEcJWBK5kjjQHIGP-NA,I never had acupuncture before. They do have S...,5,25,0,12,...,5.000000,0,0,1,1,1,0,2,2,4
1,01a4YOjiBbPUBxvgX6F7VQ,PA,3,GolOuPl1kOw_nCck2OyrgA,OVOgzUzjuqHLdwUNofsz9w,Been there twice so far and have to say Maggie...,5,7,0,10,...,5.000000,0,0,1,1,1,0,2,2,4
2,04ogq23JnzauhezrWEXG_g,PA,48,unnvLojxXbPhbqsq39GRIw,9udpA-IU2BQY-JTtM3cAxw,Mmmm...BBQ.\r\r\n\r\r\n\r\r\nI am a sucker for...,5,22,0,72,...,3.927011,0,0,0,0,1,3,9,12,41
3,04ogq23JnzauhezrWEXG_g,PA,48,1YPFrlSsnab1jTtRdQEWjw,h6ylqst_kpul0Gl7Pvu2Mw,I passed this place on the way to work everyda...,4,26,1,73,...,3.927011,0,0,0,0,1,3,9,12,41
4,04ogq23JnzauhezrWEXG_g,PA,48,ZG9QD6MJhV-M61dFDOUuBw,KulLATnwy9uSgasez-NJQw,I am loving this place! The brisket is smokey...,5,1,0,0,...,3.927011,0,0,0,0,1,3,9,12,41
5,04ogq23JnzauhezrWEXG_g,PA,48,2BYGvFAgV87LgIk4EqLtwg,ln6vnKX0rpHBqavlfejQmA,The best part about this place is the smoky sm...,3,71,0,23,...,3.927011,0,1,1,1,1,3,9,12,41
6,04ogq23JnzauhezrWEXG_g,PA,48,3CxWftKJAqpsYRW0FqIWeg,h1zm4gMClF8R_kkQPNba7w,"Honestly, there is no better BBQ in the burgh....",5,87,3,173,...,3.927011,0,0,0,0,1,3,9,12,41
7,04ogq23JnzauhezrWEXG_g,PA,48,k7wYU3nGV7u8ToIye_mCfw,v1wypJt0LvUpxi6N2PWYg,"Good BBQ, vinegar based BBQ aside I def liked ...",4,73,0,70,...,3.927011,0,0,1,1,1,3,9,12,41
8,04ogq23JnzauhezrWEXG_g,PA,48,ssd4Ac6OoABXZIai_N5Odw,jexIzwhX0l31lrQpNX3NqA,Simply awesome!\r\r\n\r\r\nWe went here for lu...,5,143,0,193,...,3.927011,0,0,0,0,1,3,9,12,41
9,04ogq23JnzauhezrWEXG_g,PA,48,AEQowxQ8076LsWNirwHTww,8Di2mqdGEphmfZrXz5ALIw,Really? This is what yinzers consider good BBQ...,2,61,1,14,...,3.927011,0,0,1,1,1,3,9,12,41


In [1497]:
join

,business_id,categories,city,state,name,review_count,stars,latitude,longitude,attributes[Alcohol],...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,shrMOQmX_SX-KualwYoAzA,"[Bakeries, Food]",Pittsburgh,PA,Au Bon Pain Co,7,3.0,40.441352,-79.998017,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,[Downtown],True
1,c3GRR8q1h4fdNV7APLzSeQ,"[Auto Repair, Automotive, Tires, Oil Change St...",Pittsburgh,PA,Homer's Service Center,40,4.5,40.423409,-79.928138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Greenfield],True
2,KxidRGk6RpyvVXibiy_DIg,"[Shopping, Home Services, Building Supplies, H...",Carnegie,PA,Lowe's,7,3.0,40.387855,-80.094976,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True
3,OT3yVpDrDVITDCIYox8d9A,"[Restaurants, Italian]",Pittsburgh,PA,Buon Giorno Cafe,5,3.5,40.438660,-80.005135,NaN,...,False,False,False,False,False,False,False,True,[Downtown],True
4,Verns0y0NTYj96SyvKgnhA,"[Optometrists, Ophthalmologists, Health & Medi...",Pittsburgh,PA,UPMC Eye Center,6,1.5,40.440987,-79.957777,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True
5,ysCLY934sncfn_4OByGhNg,"[Hair Salons, Beauty & Spas]",Pittsburgh,PA,Mint Salon & Gallery,22,4.5,40.456731,-79.930347,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Shadyside],True
6,osixLFHD7OKEUy2HSbN-7Q,"[Food, Coffee & Tea]",Pittsburgh,PA,Delanie's Coffee,74,4.5,40.428990,-79.980749,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[South Side],True
7,rXb8s7TOx2-uWxzACEXcpg,"[American (New), Restaurants]",Pittsburgh,PA,Six Penn Kitchen,216,3.0,40.442799,-80.002503,full_bar,...,False,False,False,False,False,False,False,False,[Downtown],True
8,51QMwutUI352Gmb3t1aryw,"[American (Traditional), Sandwiches, Restaurants]",McKees Rocks,PA,Rockefeller's Grille,17,3.5,40.468213,-80.094956,full_bar,...,False,False,False,False,False,False,False,False,[],True
9,TSFapUMwdyyxoi2egkOhw,"[Steakhouses, Restaurants]",Pittsburgh,PA,Sammy's Famous Corned Beef,8,2.5,40.439202,-79.998948,beer_and_wine,...,False,False,False,False,False,False,False,False,[Downtown],True


In [999]:
cols = join_tmp.columns.tolist()
cols

['business_id',
 'state',
 'business_review_count',
 'user_id',
 'review_id',
 'text',
 'stars_review',
 'user_review_count',
 'fans',
 'votes[useful]',
 'yelping_since',
 'date',
 'year_weighted',
 'review_count_weighted',
 'fans_weighted',
 'votes_weighted',
 'weighted_users',
 'weighted_rating',
 'users_sum',
 'ratings_sum',
 'ratings_avg',
 'after_Jun2015',
 'after_Jan2015',
 'after_Jun2014',
 'after_Jan2014',
 'after_Jan2010',
 'after_Jan2015_count',
 'after_Jun2014_count',
 'after_Jan2014_count',
 'after_Jan2010_count']

In [1498]:
users_sum_grouped=join_tmp.groupby(['business_id'])['users_sum'].min().reset_index(name='users_sum')
right=join
left=users_sum_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

ratings_avg_grouped=join_tmp.groupby(['business_id'])['ratings_avg'].min().reset_index(name='ratings_avg')
right=join
left=ratings_avg_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [1499]:
jan2015_grouped=join_tmp.groupby(['business_id'])['after_Jan2015'].min().reset_index(name='after_Jan2015_count')
right=join
left=after_Jan2015_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

jun2014_grouped=join_tmp.groupby(['business_id'])['after_Jun2014'].min().reset_index(name='after_Jun2014_count')
right=join
left=after_Jun2014_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

jan2014_grouped=join_tmp.groupby(['business_id'])['after_Jan2014'].min().reset_index(name='after_Jan2014_count')
right=join
left=after_Jan2014_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

jan2010_grouped=join_tmp.groupby(['business_id'])['after_Jan2010'].min().reset_index(name='after_Jan2010_count')
right=join
left=after_Jan2010_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [1500]:
join

,business_id,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg,users_sum,categories,city,state,...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,01a4YOjiBbPUBxvgX6F7VQ,4,2,2,0,5.000000,1.810000,"[Acupuncture, Traditional Chinese Medicine, He...",Pittsburgh,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True
1,04ogq23JnzauhezrWEXG_g,41,12,9,3,4.063206,58.275000,"[Caterers, Event Planning & Services, Barbeque...",Pittsburgh,PA,...,False,False,False,False,False,False,False,False,[Banksville],True
2,06SQoq5EV4XAY-yKrEtCQg,4,1,0,0,4.324182,7.588333,"[Hotels & Travel, Car Rental]",Pittsburgh,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True
3,079g3KpS_7j_Zw53JmD9Vg,67,20,14,5,3.857020,115.866667,"[American (Traditional), Restaurants]",Pittsburgh,PA,...,False,False,False,False,False,True,False,False,[Shadyside],True
4,08Z2A6u4kmaAZYeecwA4qw,18,7,6,5,4.916874,12.030000,"[Arts & Entertainment, Art Galleries, Shopping]",Pittsburgh,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
5,08arvIeYVIi4leIuFGhXBw,16,11,10,5,4.797668,19.300000,"[Tires, Automotive]",Pittsburgh,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Bellevue],True
6,08lbwQLMLaZNHRGIo5WJtQ,7,7,6,1,2.961170,23.306667,"[Shopping, Drugstores]",Pittsburgh,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True
7,09r7xwM1OTEM2hnAM8WPFA,42,17,16,7,2.993569,53.383333,"[Italian, Comfort Food, Asian Fusion, Restaura...",Oakland,PA,...,False,False,False,False,False,False,False,False,[Oakland],True
8,0DlGOTI4xXLgF2eEm_E82Q,29,10,6,2,3.732660,52.143333,"[Hotels & Travel, Event Planning & Services, H...",Pittsburgh,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
9,0E72Z_84LMrqLFAEJ276pA,3,0,0,0,4.500000,1.490000,"[Tobacco Shops, Shopping]",Verona,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True


In [1501]:
join.to_csv("traindata_newfeatures.csv", encoding='utf-8')

In [1222]:
join.to_csv("testdata_newfeatures.csv", encoding='utf-8')

In [1722]:
PA_clustered = pd.read_csv('PA_clustered.csv')
PA_clustered = PA_clustered.drop('Unnamed: 0', 1)

In [1723]:
join = pd.read_csv('traindata_newfeatures.csv')
join = join.drop('Unnamed: 0', 1)

In [1724]:
PA_clustered

,business_id,lat,lon,clusters
0,01a4YOjiBbPUBxvgX6F7VQ,40.381770,-80.045186,1
1,04ogq23JnzauhezrWEXG_g,40.421516,-80.029730,2
2,06SQoq5EV4XAY-yKrEtCQg,40.443227,-80.004835,3
3,079g3KpS_7j_Zw53JmD9Vg,40.451096,-79.933261,4
4,08Z2A6u4kmaAZYeecwA4qw,40.458077,-80.009805,5
5,08arvIeYVIi4leIuFGhXBw,40.486936,-80.054628,6
6,08lbwQLMLaZNHRGIo5WJtQ,40.451886,-79.951931,7
7,09r7xwM1OTEM2hnAM8WPFA,40.441879,-79.956889,8
8,0DlGOTI4xXLgF2eEm_E82Q,40.448564,-80.004616,9
9,0E72Z_84LMrqLFAEJ276pA,40.481752,-79.850489,10


In [1725]:
right=join
left=PA_clustered
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [1726]:
cols = join.columns.tolist()
cols

['business_id',
 'lat',
 'lon',
 'clusters',
 'after_Jan2010_count',
 'after_Jan2014_count',
 'after_Jun2014_count',
 'after_Jan2015_count',
 'ratings_avg',
 'users_sum',
 'categories',
 'city',
 'state',
 'name',
 'review_count',
 'stars',
 'latitude',
 'longitude',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open']

In [1727]:
join=join[['business_id',
 'clusters',
 'after_Jan2010_count',
 'after_Jan2014_count',
 'after_Jun2014_count',
 'after_Jan2015_count',
 'ratings_avg',
 'users_sum',
 'categories',
 'city',
 'state',
 'name',
 'review_count',
 'stars',
 'latitude',
 'longitude',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open']]

In [1728]:
join

,business_id,clusters,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg,users_sum,categories,city,...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,01a4YOjiBbPUBxvgX6F7VQ,1,4,2,2,0,5.000000,1.810000,"[Acupuncture, Traditional Chinese Medicine, He...",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True
1,04ogq23JnzauhezrWEXG_g,2,41,12,9,3,4.063206,58.275000,"[Caterers, Event Planning & Services, Barbeque...",Pittsburgh,...,False,False,False,False,False,False,False,False,[Banksville],True
2,06SQoq5EV4XAY-yKrEtCQg,3,4,1,0,0,4.324182,7.588333,"[Hotels & Travel, Car Rental]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True
3,079g3KpS_7j_Zw53JmD9Vg,4,67,20,14,5,3.857020,115.866667,"[American (Traditional), Restaurants]",Pittsburgh,...,False,False,False,False,False,True,False,False,[Shadyside],True
4,08Z2A6u4kmaAZYeecwA4qw,5,18,7,6,5,4.916874,12.030000,"[Arts & Entertainment, Art Galleries, Shopping]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
5,08arvIeYVIi4leIuFGhXBw,6,16,11,10,5,4.797668,19.300000,"[Tires, Automotive]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Bellevue],True
6,08lbwQLMLaZNHRGIo5WJtQ,7,7,7,6,1,2.961170,23.306667,"[Shopping, Drugstores]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True
7,09r7xwM1OTEM2hnAM8WPFA,8,42,17,16,7,2.993569,53.383333,"[Italian, Comfort Food, Asian Fusion, Restaura...",Oakland,...,False,False,False,False,False,False,False,False,[Oakland],True
8,0DlGOTI4xXLgF2eEm_E82Q,9,29,10,6,2,3.732660,52.143333,"[Hotels & Travel, Event Planning & Services, H...",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
9,0E72Z_84LMrqLFAEJ276pA,10,3,0,0,0,4.500000,1.490000,"[Tobacco Shops, Shopping]",Verona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True


In [1737]:
user_rating_set=trainingdata[['business_id',
 'users_sum',
 'ratings_avg','ratings_sum']]

In [1738]:
user_rating_set

,business_id,users_sum,ratings_avg,ratings_sum
0,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
1,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
2,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
3,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
4,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
5,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
6,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
7,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
8,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667
9,UE_y6auTgq3FXlvUMkbw,160.901667,3.301872,531.276667


In [1739]:
usersumgrouped=user_rating_set.groupby(['business_id'])['users_sum'].min().reset_index(name='users_sum_after2015')

ratingsavggrouped=user_rating_set.groupby(['business_id'])['ratings_avg'].min().reset_index(name='ratings_avg_after2015')

ratingsumavggrouped=user_rating_set.groupby(['business_id'])['ratings_sum'].min().reset_index(name='ratings_sum_after2015')

join_new_regression_value=join

In [1740]:
right=join_new_regression_value
left=usersumgrouped
new_regression_value_merged=left.merge(right, left_on='business_id', right_on='business_id', how='inner')


In [1742]:
right=new_regression_value_merged
left=ratingsumavggrouped
new_regression_value_merged=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

right=new_regression_value_merged
left=ratingsavggrouped
new_regression_value_merged=left.merge(right, left_on='business_id', right_on='business_id', how='inner')



In [1743]:
new_regression_value_merged

,business_id,ratings_avg_after2015,ratings_sum_after2015,users_sum_after2015,clusters,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg,...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,01a4YOjiBbPUBxvgX6F7VQ,5.000000,19.050000,3.810000,1,4,2,2,0,5.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True
1,04ogq23JnzauhezrWEXG_g,3.927011,268.116667,68.275000,2,41,12,9,3,4.063206,...,False,False,False,False,False,False,False,False,[Banksville],True
2,06SQoq5EV4XAY-yKrEtCQg,3.937124,33.813333,8.588333,3,4,1,0,0,4.324182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True
3,079g3KpS_7j_Zw53JmD9Vg,3.817627,521.233333,136.533333,4,67,20,14,5,3.857020,...,False,False,False,False,False,True,False,False,[Shadyside],True
4,08Z2A6u4kmaAZYeecwA4qw,4.970899,170.816667,34.363333,5,18,7,6,5,4.916874,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
5,08arvIeYVIi4leIuFGhXBw,4.628873,98.595000,21.300000,6,16,11,10,5,4.797668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Bellevue],True
6,08lbwQLMLaZNHRGIo5WJtQ,2.961170,69.015000,23.306667,7,7,7,6,1,2.961170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True
7,09r7xwM1OTEM2hnAM8WPFA,3.028215,177.806667,58.716667,8,42,17,16,7,2.993569,...,False,False,False,False,False,False,False,False,[Oakland],True
8,0DlGOTI4xXLgF2eEm_E82Q,3.672476,208.633333,56.810000,9,29,10,6,2,3.732660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
9,0E72Z_84LMrqLFAEJ276pA,4.700803,11.705000,2.490000,10,3,0,0,0,4.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True


In [1746]:
new_regression_value_merged.to_csv("traindata_newfeatures_clustered.csv", encoding='utf-8')

In [1730]:
category_list


{}

In [37]:
reg_op = pd.read_csv('regression_model_op.csv')
reg_op = reg_op.drop('Unnamed: 0', 1)
reg_op = reg_op.drop('X', 1)

In [38]:
reg_op

,business_id,ratings_avg_after2015,ratings_sum_after2015,users_sum_after2015,clusters,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,regression_value,...,attributes.Ambience..hipster.,attributes.Ambience..intimate.,attributes.Ambience..romantic.,attributes.Ambience..touristy.,attributes.Ambience..trendy.,attributes.Ambience..upscale.,attributes.Delivery.,neighborhoods,open,predicted_value
0,01a4YOjiBbPUBxvgX6F7VQ,5.000000,19.050000,3.810000,1,4,2,2,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True,19.583750
1,04ogq23JnzauhezrWEXG_g,3.927011,268.116667,68.275000,2,41,12,9,3,192.0,...,False,False,False,False,False,False,False,[Banksville],True,196.081600
2,06SQoq5EV4XAY-yKrEtCQg,3.937124,33.813333,8.588333,3,4,1,0,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True,18.059617
3,079g3KpS_7j_Zw53JmD9Vg,3.817627,521.233333,136.533333,4,67,20,14,5,259.0,...,False,False,False,False,True,False,False,[Shadyside],True,281.023517
4,08Z2A6u4kmaAZYeecwA4qw,4.970899,170.816667,34.363333,5,18,7,6,5,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True,89.610983
5,08arvIeYVIi4leIuFGhXBw,4.628873,98.595000,21.300000,6,16,11,10,5,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Bellevue],True,75.805067
6,08lbwQLMLaZNHRGIo5WJtQ,2.961170,69.015000,23.306667,7,7,7,6,1,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True,26.203033
7,09r7xwM1OTEM2hnAM8WPFA,3.028215,177.806667,58.716667,8,42,17,16,7,150.0,...,False,False,False,False,False,False,False,[Oakland],True,154.687033
8,0DlGOTI4xXLgF2eEm_E82Q,3.672476,208.633333,56.810000,9,29,10,6,2,115.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True,117.706700
9,0E72Z_84LMrqLFAEJ276pA,4.700803,11.705000,2.490000,10,3,0,0,0,13.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],True,13.478417


In [40]:
def category_ratings(reg_op):
    negcluster = reg_op[reg_op['ratings_avg']<3.5].reset_index()
    poscluster = reg_op[reg_op['ratings_avg']>=3.5].reset_index()

    pncaty={}

    for i in range(len(poscluster['categories'])):
        foodlist = poscluster['categories'][i].strip("[]").split(',')
        wt = poscluster['predicted_value'][i]
        for word in foodlist:
            word = word.rstrip(',')
            pncaty[word]=pncaty.get(word,0)+wt
            #print word


    for i in range(len(negcluster['categories'])):
        foodlist = negcluster['categories'][i].strip("[]").split()
        wt = -(negcluster['predicted_value'][i])
        for word in foodlist:
            word = word.rstrip(',')
            pncaty[word]=pncaty.get(word,0)+wt
    return pncaty

In [41]:
clusters_grouped=reg_op.groupby(['clusters']).apply(category_ratings)

In [42]:
def category_frequencies(reg_op):
    poscluster = reg_op.reset_index()
    pncaty={}

    for i in range(len(poscluster['categories'])):
        foodlist = poscluster['categories'][i].strip("[]").split(',')
        for word in foodlist:
            word = word.rstrip(',')
            pncaty[word]=pncaty.get(word,0)+1
    #print word
    #sorted_words = (sorted(pncaty.items(), key=operator.itemgetter(1), reverse = True))        
    return pncaty

In [43]:
clusters_rating = reg_op.groupby(['clusters']).apply(category_ratings)


In [44]:
clusters_frequency = reg_op.groupby(['clusters']).apply(category_frequencies)


In [45]:
clusters_frequency

clusters
1      {u' Ice Cream & Frozen Yogurt': 2, u' Shopping...
2      {u' Shopping': 1, u'Hotels & Travel': 1, u'Sho...
3      {u' Auto Repair': 1, u' Candy Stores': 2, u' V...
4      {u'': 1, u' Candy Stores': 1, u' Dentists': 1,...
5      {u'Caribbean': 1, u' Shopping': 1, u' Hot Dogs...
6      {u' Ice Cream & Frozen Yogurt': 1, u' Shopping...
7      {u' Dentists': 2, u' Piercing': 1, u'Arts & En...
8      {u' Auto Repair': 1, u' Ophthalmologists': 1, ...
9      {u' Stadiums & Arenas': 1, u'Chicken Wings': 1...
10     {u' Shopping': 1, u' Nail Salons': 1, u'Beauty...
11     {u' Gas & Service Stations': 1, u' Dentists': ...
12     {u' Candy Stores': 1, u' Dentists': 4, u'Arts ...
13     {u' Sports Bars': 1, u' Formal Wear': 1, u' Sh...
14     {u' Dentists': 2, u'Arts & Entertainment': 2, ...
15     {u'Fondue': 1, u'Arts & Entertainment': 1, u' ...
16     {u' Candy Stores': 1, u' Dentists': 1, u' Musi...
17     {u'': 1, u'Arts & Entertainment': 1, u'Sandwic...
18     {u' Candy Store

In [47]:
clusters_rating

clusters
1      {u'Salons': -9.3644, u' Ice Cream & Frozen Yog...
2      {u'(New)': -67.6524, u'Caterers': 196.0816, u'...
3      {u'Vitamins': -19.5279, u'Drugstores': -21.719...
4      {u'': 56.2779, u' Candy Stores': 20.612, u' De...
5      {u'Caribbean': 50.26495, u' Shopping': 89.6109...
6      {u'Salons': -25.5126833333, u' Ice Cream & Fro...
7      {u'Salons': -186.21615, u' Dentists': 30.32305...
8      {u'Universities': -8.17995, u'Notaries': -11.3...
9      {u' Stadiums & Arenas': 1703.62788333, u'Arts'...
10     {u' Shopping': 13.4784166667, u' Nail Salons':...
11     {u' Dentists': 16.4369166667, u'Service': -4.9...
12     {u' Candy Stores': 24.1302, u' Dentists': 103....
13     {u'Salons': -9.13158333333, u'Arts': -40.49683...
14     {u' Dentists': 11.65185, u'Arts & Entertainmen...
15     {u'Service': -8.73798333333, u'Fondue': 331.65...
16     {u' Candy Stores': 28.5791333333, u' Musical I...
17     {u'Salons': -11.6716666667, u'Service': -11.12...
18     {u' Candy Store

In [16]:
clusters_rating_cats =  pd.Series.to_frame(clusters_rating)
clusters_rating_cats = clusters_rating_cats[0].reset_index()
clusters_rating_cats.columns = ['cluster', 'cluster_ratings']

In [17]:
clusters_new_ratings=clusters_rating_cats

In [18]:
clusters_rating_cats

,cluster,cluster_ratings
0,1,"{u'Salons': -9.3644, u' Ice Cream & Frozen Yog..."
1,2,"{u'(New)': -67.6524, u'Caterers': 196.0816, u'..."
2,3,"{u'Vitamins': -19.5279, u'Drugstores': -21.719..."
3,4,"{u'': 56.2779, u' Candy Stores': 20.612, u' De..."
4,5,"{u'Caribbean': 50.26495, u' Shopping': 89.6109..."
5,6,"{u'Salons': -25.5126833333, u' Ice Cream & Fro..."
6,7,"{u'Salons': -186.21615, u' Dentists': 30.32305..."
7,8,"{u'Universities': -8.17995, u'Notaries': -11.3..."
8,9,"{u' Stadiums & Arenas': 1703.62788333, u'Arts'..."
9,10,"{u' Shopping': 13.4784166667, u' Nail Salons':..."


In [19]:
clusters_frequency_cats =  pd.Series.to_frame(clusters_frequency)
clusters_frequency_cats = clusters_frequency_cats[0].reset_index()
clusters_frequency_cats.columns = ['cluster', 'cluster_frequency']
clusters_new_freq=clusters_frequency_cats

In [20]:
clusters_new_freq

,cluster,cluster_frequency
0,1,"{u' Ice Cream & Frozen Yogurt': 2, u' Shopping..."
1,2,"{u' Shopping': 1, u'Hotels & Travel': 1, u'Sho..."
2,3,"{u' Auto Repair': 1, u' Candy Stores': 2, u' V..."
3,4,"{u'': 1, u' Candy Stores': 1, u' Dentists': 1,..."
4,5,"{u'Caribbean': 1, u' Shopping': 1, u' Hot Dogs..."
5,6,"{u' Ice Cream & Frozen Yogurt': 1, u' Shopping..."
6,7,"{u' Dentists': 2, u' Piercing': 1, u'Arts & En..."
7,8,"{u' Auto Repair': 1, u' Ophthalmologists': 1, ..."
8,9,"{u' Stadiums & Arenas': 1, u'Chicken Wings': 1..."
9,10,"{u' Shopping': 1, u' Nail Salons': 1, u'Beauty..."


In [21]:
freq_list=[]
freq_top_sorted=[]
freq_low_sorted=[]

for i in range(len(clusters_new_freq['cluster'])):
    words = (sorted(clusters_new_freq['cluster_frequency'][i].items(), key=operator.itemgetter(1), reverse = True))
    freq_top_sorted.append(words)
    low_words=(sorted(clusters_new_freq['cluster_frequency'][i].items(), key=operator.itemgetter(1)))
    freq_low_sorted.append(low_words)
    wordlist=[]
    for word in words:
        wordlist.append(word[0])
        if(len(wordlist)>9):
            break
    freq_list.append(wordlist)

In [22]:
freq_list

[[' Restaurants',
  ' Shopping',
  'Food',
  ' Beauty & Spas',
  ' Health & Medical',
  'Hair Salons',
  ' Nail Salons',
  ' Coffee & Tea',
  ' Ice Cream & Frozen Yogurt',
  'American (New)'],
 [' Restaurants',
  'Indian',
  ' Shopping',
  'Hotels & Travel',
  'Shopping Centers',
  ' American (New)',
  ' Delis',
  'American (Traditional)',
  ' Caterers',
  'Caterers'],
 [' Restaurants',
  'Food',
  ' Shopping',
  ' Nightlife',
  'Bars',
  ' Coffee & Tea',
  ' Italian',
  ' Sandwiches',
  'Hotels & Travel',
  'Restaurants'],
 [' Restaurants',
  ' Shopping',
  'Food',
  ' Fashion',
  ' Beauty & Spas',
  'Shopping',
  "Women's Clothing",
  'Bars',
  ' Nightlife',
  ' Event Planning & Services'],
 [' Restaurants',
  ' Chicken Wings',
  'Food',
  'Caribbean',
  ' Shopping',
  ' Hot Dogs',
  'Arts & Entertainment',
  ' Mass Media',
  ' Eyelash Service',
  'Pizza'],
 [' Restaurants',
  'Food',
  ' Coffee & Tea',
  'Pizza',
  ' Shopping',
  ' Automotive',
  ' Food',
  'Beauty & Spas',
  ' Beau

In [23]:
clusters_frequency_cats['top_freq']="[]"
for i in range(len(clusters_frequency_cats['cluster'])):
       clusters_frequency_cats['top_freq'][i] = freq_list[i]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:
clusters_frequency_cats.head()

,cluster,cluster_frequency,top_freq
0,1,"{u' Ice Cream & Frozen Yogurt': 2, u' Shopping...","[ Restaurants, Shopping, Food, Beauty & Spas..."
1,2,"{u' Shopping': 1, u'Hotels & Travel': 1, u'Sho...","[ Restaurants, Indian, Shopping, Hotels & Tra..."
2,3,"{u' Auto Repair': 1, u' Candy Stores': 2, u' V...","[ Restaurants, Food, Shopping, Nightlife, Ba..."
3,4,"{u'': 1, u' Candy Stores': 1, u' Dentists': 1,...","[ Restaurants, Shopping, Food, Fashion, Bea..."
4,5,"{u'Caribbean': 1, u' Shopping': 1, u' Hot Dogs...","[ Restaurants, Chicken Wings, Food, Caribbean..."


In [25]:
rating_list=[]
rating_top_sorted=[]
rating_low_sorted=[]

for i in range(len(clusters_new_ratings['cluster'])):
    words = (sorted(clusters_new_ratings['cluster_ratings'][i].items(), key=operator.itemgetter(1), reverse = True))
    rating_top_sorted.append(words)
    low_words=(sorted(clusters_new_ratings['cluster_ratings'][i].items(), key=operator.itemgetter(1)))
    rating_low_sorted.append(low_words)
    wordlist=[]
    for word in words:
        wordlist.append(word[0])
        if(len(wordlist)>9):
            break
    rating_list.append(wordlist)

In [26]:
rating_list

[[' Restaurants',
  'Sushi Bars',
  ' Pizza',
  'Italian',
  ' Japanese',
  ' Chinese',
  'Pizza',
  'American (New)',
  'Moroccan',
  ' Sandwiches'],
 [' Restaurants',
  'Pizza',
  'Caterers',
  ' Event Planning & Services',
  ' Barbeque',
  ' Party & Event Planning',
  ' Caterers',
  'Event Planning & Services',
  'Home Services',
  ' Heating & Air Conditioning/HVAC'],
 [' Restaurants',
  ' American (New)',
  'American (New)',
  ' Nightlife',
  ' Italian',
  ' Pizza',
  'Steakhouses',
  ' Sandwiches',
  'Hotels & Travel',
  ' Event Planning & Services'],
 [' Restaurants',
  'Food',
  'Breakfast & Brunch',
  ' Pizza',
  ' Italian',
  ' Gelato',
  ' Beauty & Spas',
  ' Shopping',
  ' Food',
  ' American (New)'],
 [' Restaurants',
  'Mexican',
  ' Shopping',
  'Arts & Entertainment',
  ' Art Galleries',
  'Food',
  'Caribbean',
  ' Chicken Wings',
  'Seafood',
  ' Eyelash Service'],
 [' Restaurants',
  'Food',
  ' Coffee & Tea',
  ' Bagels',
  ' Juice Bars & Smoothies',
  'Pizza',
  'Tu

In [27]:
clusters_rating_cats['top_categories']="[]"
for i in range(len(clusters_rating_cats['cluster'])):
       clusters_rating_cats['top_categories'][i] = rating_list[i]


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [28]:
clusters_rating_cats.head()

,cluster,cluster_ratings,top_categories
0,1,"{u'Salons': -9.3644, u' Ice Cream & Frozen Yog...","[ Restaurants, Sushi Bars, Pizza, Italian, J..."
1,2,"{u'(New)': -67.6524, u'Caterers': 196.0816, u'...","[ Restaurants, Pizza, Caterers, Event Plannin..."
2,3,"{u'Vitamins': -19.5279, u'Drugstores': -21.719...","[ Restaurants, American (New), American (New)..."
3,4,"{u'': 56.2779, u' Candy Stores': 20.612, u' De...","[ Restaurants, Food, Breakfast & Brunch, Pizz..."
4,5,"{u'Caribbean': 50.26495, u' Shopping': 89.6109...","[ Restaurants, Mexican, Shopping, Arts & Ente..."


In [29]:
top_rated_businesses_clustered=clusters_rating_cats[['top_categories']]


In [1794]:
top_rated_businesses_clustered.to_csv("top_rated_businesses_clustered.csv", encoding='utf-8')

In [48]:
mined_rules = pd.read_csv('arules_op.csv')
mined_rules


,Unnamed: 0,lhs,Unnamed: 2,rhs,support,confidence,lift
0,1,Bars,=>,Nightlife,0.022059,1.00,2.956522
1,2,Active,=>,Life,0.022059,1.00,45.333333
2,3,Life,=>,Active,0.022059,1.00,45.333333
3,4,Bars,=>,Nightlife,0.022059,1.00,2.956522
4,7,Bakeries,=>,Food,0.022059,1.00,2.518519
5,8,Caterers,=>,Event Planning & Services,0.022059,1.00,7.555556
6,9,Caterers,=>,Restaurants,0.022059,1.00,1.619048
7,10,Caribbean,=>,Restaurants,0.022059,1.00,1.619048
8,11,Veterinarians,=>,Pets,0.022059,1.00,17.000000
9,12,Thai,=>,Food,0.022059,1.00,2.518519


In [31]:
mined_rules=mined_rules[['lhs','rhs']]
mined_rules

,lhs,rhs
0,Bars,Nightlife
1,Active,Life
2,Life,Active
3,Bars,Nightlife
4,Bakeries,Food
5,Caterers,Event Planning & Services
6,Caterers,Restaurants
7,Caribbean,Restaurants
8,Veterinarians,Pets
9,Thai,Food


In [49]:
rule_dict={}

for i in range(len(mined_rules['lhs'])):
    key = mined_rules['rhs'][i].strip()
    
    if key not in rule_dict:
        values =  []
        rule_dict[key] = values
    else:
        values = rule_dict[key]
        
    for word in mined_rules['lhs'][i].split(','):
        word = word.strip()
        if word not in values:
            values.append(word)

opportunities = {}
for key in rule_dict.keys():
    cid={}
    for i in range(len(freq_list)):
        if key in freq_list[i]:
            rule_values = rule_dict[key]
            freq_values = freq_list[i]
            caty = []
            for rule_value in rule_values:
                if rule_value not in freq_values:
                    caty.append(rule_value)            
            cid[i]=caty
    
    opportunities[key] = cid

In [50]:
opportunities

{'Active': {},
 'American (New)': {0: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife',
   'Pizza'],
  8: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife',
   'Pizza'],
  18: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife',
   'Pizza'],
  20: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife',
   'Pizza'],
  29: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife'],
  32: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife'],
  35: ['Steakhouses', 'Event Planning & Services', 'Italian', 'Nightlife'],
  43: ['Steakhouses',
   'Restaurants',
   'Event Planning & Services',
   'Italian',
   'Nightlife'],
  52: ['Steakhouses', 'Event Planning & Services', 'Italian', 'Nightlife'],
  64: ['Steakhouses',
   'Restaurants',
   'Even

In [34]:
business_opportunities=pd.DataFrame(opportunities.items())
business_opportunities.columns = ['business_category', 'potential_opportunities']

In [35]:
business_opportunities.head()

,business_category,potential_opportunities
0,American (New),"{0: [u'Steakhouses', u'Restaurants', u'Event P..."
1,Beauty & Spas,"{32: [u'Hair Salons', u'Day Spas', u'Nail Salo..."
2,Arts & Entertainment,"{65: [u'Art Galleries', u'Stadiums & Arenas', ..."
3,Breakfast & Brunch,"{107: [u'Diners', u'Ice Cream & Frozen Yogurt'..."
4,Beer,{}


In [36]:
business_opportunities.to_csv("business_opportunities.csv", encoding='utf-8')